# Train and deploy with Cloud AI Platform

In [1]:
import json
import os
import pandas as pd
import time
import googleapiclient.discovery

In [54]:
TRAINING_APP_FOLDER = '../training_app/trainer'
PREDICT_APP_FOLDER = '../predict_app'
TRAINING_DATA_PATH = 'gs://jk-demo-datasets/gasdata/training.csv'
TESTING_DATA_PATH = 'gs://jk-demo-datasets/gasdata/testing.csv'
REGION = "us-central1"
ARTIFACT_BUCKET = "gs://jk-demo-artifacts"

## Create a training application

In [4]:
os.makedirs(TRAINING_APP_FOLDER, exist_ok=True)
!touch $TRAINING_APP_FOLDER/__init__.py

### Create a training script

In [5]:
%%writefile $TRAINING_APP_FOLDER/train.py

import logging
import os
import subprocess
import sys

import fire
import numpy as np
import pandas as pd

from sklearn.externals import joblib
from sklearn.cross_decomposition import PLSRegression
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import Ridge
from sklearn.manifold import TSNE 
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


def train(job_dir, data_path, n_features_options, l2_reg_options):
    
  # Load data from GCS
  df_train = pd.read_csv(data_path)

  y = df_train.octane
  X = df_train.drop('octane', axis=1)
    
  pipeline = Pipeline([
    ('scale', StandardScaler()),
    ('reduce_dim', PCA()),
    ('regress', Ridge())
  ])

  param_grid = [
    {
      'reduce_dim__n_components': n_features_options,
      'regress__alpha': l2_reg_options
    }
  ]

  grid = GridSearchCV(pipeline, cv=10, n_jobs=None, param_grid=param_grid, scoring='neg_mean_squared_error', iid=False)
  
  grid.fit(X, y)

  logging.info("Best estimator: {}".format(grid.best_params_))
  logging.info("Best score: {}".format(grid.best_score_))
    
  # Retrain the best model on a full dataset
  best_estimator = grid.best_estimator_
  trained_pipeline = best_estimator.fit(X, y)

  # Save the model
  model_filename = 'model.joblib'
  joblib.dump(value=trained_pipeline, filename=model_filename)
  gcs_model_path = "{}/trained_model/{}".format(job_dir, model_filename)
  subprocess.check_call(['gsutil', 'cp', model_filename, gcs_model_path], stderr=sys.stdout)
  logging.info("Saved model in: {}".format(gcs_model_path)) 
    
if __name__ == "__main__":
  logging.basicConfig(level=logging.INFO)
  fire.Fire(train)

Writing ../training_app/trainer/train.py


### Configure dependencies

In [6]:
%%writefile $TRAINING_APP_FOLDER/../setup.py

from setuptools import find_packages
from setuptools import setup

REQUIRED_PACKAGES = ['fire', 'gcsfs']

setup(
    name='trainer',
    version='0.1',
    install_requires=REQUIRED_PACKAGES,
    packages=find_packages(),
    include_package_data=True,
    description='My training application package.'
)


Writing ../training_app/trainer/../setup.py


## Submit a training job

In [7]:
JOB_NAME = "JOB_{}".format(time.strftime("%Y%m%d_%H%M%S"))
SCALE_TIER = "BASIC"
#JOB_DIR = "gs://jk-demo-jobdir/{}".format(JOB_NAME)
MODULE_NAME = "trainer.train"
RUNTIME_VERSION = "1.14"
PYTHON_VERSION = "3.5"

N_FEATURES_OPTIONS="[2,4,6]"
L2_REG_OPTIONS="[0.1,0.2,0.3,0.5]"

In [8]:
!gcloud ai-platform jobs submit training $JOB_NAME \
--region $REGION \
--job-dir $ARTIFACT_BUCKET/jobs/$JOB_NAME \
--package-path $TRAINING_APP_FOLDER \
--module-name $MODULE_NAME \
--scale-tier $SCALE_TIER \
--python-version $PYTHON_VERSION \
--runtime-version $RUNTIME_VERSION \
-- \
--data_path $TRAINING_DATA_PATH \
--n_features_options $N_FEATURES_OPTIONS \
--l2_reg_options $L2_REG_OPTIONS

Job [JOB_20190820_185030] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe JOB_20190820_185030

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs JOB_20190820_185030
jobId: JOB_20190820_185030
state: QUEUED


In [9]:
!gcloud ai-platform jobs describe $JOB_NAME
!gcloud ai-platform jobs stream-logs $JOB_NAME

createTime: '2019-08-20T18:50:34Z'
etag: skWF-jPO2RI=
jobId: JOB_20190820_185030
state: PREPARING
trainingInput:
  args:
  - --data_path
  - gs://jk-demo-datasets/gasdata/training.csv
  - --n_features_options
  - '[2,4,6]'
  - --l2_reg_options
  - '[0.1,0.2,0.3,0.5]'
  jobDir: gs://jk-demo-artifacts/jobs/JOB_20190820_185030
  packageUris:
  - gs://jk-demo-artifacts/jobs/JOB_20190820_185030/packages/a2b25d0d356d4ec70cba370d0cce00dbc11690faa6f99a820264072517d6a8bf/trainer-0.1.tar.gz
  pythonModule: trainer.train
  pythonVersion: '3.5'
  region: us-west1
  runtimeVersion: '1.14'
trainingOutput: {}

View job in the Cloud Console at:
https://console.cloud.google.com/mlengine/jobs/JOB_20190820_185030?project=jk-demo1

View logs at:
https://console.cloud.google.com/logs?resource=ml.googleapis.com%2Fjob_id%2FJOB_20190820_185030&project=jk-demo1
INFO	2019-08-20 18:50:34 +0000	service		Validating job requirements...
INFO	2019-08-20 18:50:34 +0000	service		Job creation request has been successful

## Deploy using AI Platform Prediction

### Create a model resource

In [13]:
MODEL_NAME = "gas_octane_regression"
LABELS = "task=regression,domain=chemometrics"
#MODEL_PATH = "{}/trained_model/".format(JOB_DIR)
VERSION_NAME = "v01"

In [14]:
!gcloud ai-platform models create  $MODEL_NAME \
--regions=$REGION \
--labels=$LABELS

Created ml engine model [projects/jk-demo1/models/gas_octane_regression].


### Create a model version

In [23]:
!gcloud ai-platform versions create $VERSION_NAME \
--model=$MODEL_NAME \
--origin=$ARTIFACT_BUCKET/jobs/$JOB_NAME/trained_model/ \
--runtime-version=1.14 \
--framework=scikit-learn \
--python-version=3.5

Creating version (this might take a few minutes)......done.                    


### Test the model
#### Prepare a file with test instances

In [26]:
INPUT_FILE = 'instances.json'

df_test = pd.read_csv(TESTING_DATA_PATH).drop('octane', axis=1)
with open(INPUT_FILE, "w") as f:
  for index, row in df_test.iterrows():
    f.write(json.dumps(list(row.values)))
    f.write("\n")

#### Run predictions

In [27]:
!gcloud ai-platform predict \
--model $MODEL_NAME \
--version $VERSION_NAME \
--json-instances $INPUT_FILE

[83.61274117546486, 88.78144180442887, 86.60147562277632, 86.37821707761988, 84.57226241557935, 88.60871164935777]


## Deploy with a custom prediction routine
### Create a predictor class

In [28]:
os.makedirs(PREDICT_APP_FOLDER, exist_ok=True)

In [63]:
%%writefile $PREDICT_APP_FOLDER/predict.py

import os
import numpy as np

from sklearn.externals import joblib

class OctaneRegressor(object):
    """A custom prediction routine for Octane regressor"""
    
    def __init__(self, model):
        """Stores the model loaded in from_path"""
        self._model = model
        
    def predict(self, instances, **kwargs):
        """Runs inference"""
    
        outputs = self._model.predict(np.asarray(instances))
        
        return outputs.tolist()

        
    @classmethod
    def from_path(cls, model_dir):
        """Loads the model from the joblib file"""
        model_path = os.path.join(model_dir, 'model.joblib')
        model = joblib.load(model_path)
        
        
        return cls(model)
    

Overwriting ../predict_app/predict.py


### Create a source distribution package

In [64]:
%%writefile $PREDICT_APP_FOLDER/setup.py

from setuptools import setup

setup(
    name='custom-predictor',
    description='Custom prediction routine.',
    version='0.1',
    scripts=['predict.py']
)

Overwriting ../predict_app/setup.py


In [65]:
%cd $PREDICT_APP_FOLDER
%run $PREDICT_APP_FOLDER/setup.py sdist --formats=gztar
%cd -

/home/jupyter/caip-demo/predict_app
running sdist
running egg_info
writing top-level names to custom_predictor.egg-info/top_level.txt
writing dependency_links to custom_predictor.egg-info/dependency_links.txt
writing custom_predictor.egg-info/PKG-INFO
reading manifest file 'custom_predictor.egg-info/SOURCES.txt'
writing manifest file 'custom_predictor.egg-info/SOURCES.txt'


running check


creating custom-predictor-0.1
creating custom-predictor-0.1/custom_predictor.egg-info
copying files to custom-predictor-0.1...
copying predict.py -> custom-predictor-0.1
copying setup.py -> custom-predictor-0.1
copying custom_predictor.egg-info/PKG-INFO -> custom-predictor-0.1/custom_predictor.egg-info
copying custom_predictor.egg-info/SOURCES.txt -> custom-predictor-0.1/custom_predictor.egg-info
copying custom_predictor.egg-info/dependency_links.txt -> custom-predictor-0.1/custom_predictor.egg-info
copying custom_predictor.egg-info/top_level.txt -> custom-predictor-0.1/custom_predictor.egg-info
Writing custom-predictor-0.1/setup.cfg
Creating tar archive
removing 'custom-predictor-0.1' (and everything under it)
/home/jupyter/caip-demo/notebooks


### Copy the source distribution package to GCS staging area

In [66]:
VERSION_NAME = "v05"
TARBALL_NAME = "custom-predictor-0.1.tar.gz"
LOCAL_PATH = "{}/dist/{}".format(PREDICT_APP_FOLDER, TARBALL_NAME)
GCS_PATH = "{}/prediction_routines/{}/{}/{}".format(ARTIFACT_BUCKET, MODEL_NAME, VERSION_NAME, TARBALL_NAME)

In [67]:
!gsutil cp $LOCAL_PATH $GCS_PATH

Copying file://../predict_app/dist/custom-predictor-0.1.tar.gz [Content-Type=application/x-tar]...
/ [1 files][  985.0 B/  985.0 B]                                                
Operation completed over 1 objects/985.0 B.                                      


### Create model version

In [68]:
!gcloud beta ai-platform versions create $VERSION_NAME \
--model=$MODEL_NAME \
--origin=$ARTIFACT_BUCKET/jobs/$JOB_NAME/trained_model/ \
--runtime-version=1.14 \
--python-version=3.5 \
--package-uris $GCS_PATH \
--prediction-class predict.OctaneRegressor

Creating version (this might take a few minutes)......done.                    


### Test the model

In [69]:
!gcloud ai-platform predict \
--model $MODEL_NAME \
--version $VERSION_NAME \
--json-instances $INPUT_FILE

[83.61274117546486, 88.78144180442887, 86.60147562277632, 86.37821707761988, 84.57226241557935, 88.60871164935777]
